<a href="https://colab.research.google.com/github/idoFinder/RNN_LSTM_API_CALLS/blob/master/RNN_LSTM_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Sequential, load_model ,Model
from keras.layers import Dense, LSTM,Embedding,GlobalMaxPooling1D,Activation
from keras.utils import to_categorical
from keras.optimizers import SGD
from sklearn.metrics import roc_curve, precision_recall_curve, roc_auc_score, confusion_matrix, accuracy_score, recall_score,plot_confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing, metrics
from itertools import groupby
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import random
import os

Using TensorFlow backend.


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Functions

## Data

### Read raw data

In [0]:
def load_raw_parquets(path):
  all_data = pd.DataFrame(columns=['file_name','API_sequence','class'])
  for idx, batch_path in enumerate(os.listdir(path)):
      full_path = '{}/{}'.format(path,batch_path)
      batch = pd.read_parquet(full_path)
      files = batch['file_name'].unique()
      new_records = []
      for file in files:
          sub_df = batch[batch['file_name']==file].sort_values(['normalized_time'])
          file_class = sub_df.iloc[0]['class']
          new_record = {'file_name':file,'API_sequence':sub_df['API'].values,'class':file_class}
          new_records.append(new_record)
      all_data = all_data.append(new_records, ignore_index=True)
      print('batch',idx+1, ' loaded')
    
  return all_data



def get_classes_distribution_df(df):
    num_of_benign = df[df['class'] == 'benign'].shape[0]
    num_of_malware = df[df['class'] == 'malicious'].shape[0]
    total = num_of_benign + num_of_malware
    benign_perc = round((num_of_benign / total) * 100, 2)
    malware_perc = round((num_of_malware / total) * 100, 2)
    res_str = 'Benign:{} ({}%) | Malicious:{} ({}%)'.format(num_of_benign, benign_perc, num_of_malware, malware_perc)
    return res_str
  

### Data cleaning + creating API corpus


In [0]:
def remove_repitations(sequence):
    new_sequence = [x[0] for x in groupby(sequence)]
    return new_sequence


def data_cleaning(sequence_MIN_length,all_data):
  # remove repitation in all data
  all_data['API_sequence'] =all_data.apply(lambda x: remove_repitations(x['API_sequence']), axis=1)

  # remove sequences shorter then sequence_MIN_length
  print('Remove files with short sequence')
  print('files before:',all_data.shape[0])
  to_drop = []
  for file in all_data['file_name']:
      if len(all_data[all_data['file_name']==file].API_sequence.values[0]) < sequence_MIN_length:
          to_drop.append(file)
  all_data = all_data[~all_data['file_name'].isin(to_drop)]
  print('files after:',all_data.shape[0])
  all_data = all_data.reset_index(drop=True)
  return all_data


def use_spesific_files(dataset_path,all_data):
  balanced_csv = pd.read_csv(dataset_path)
  print('files before:',all_data.shape[0])
  all_data = all_data[all_data['file_name'].isin(balanced_csv['file_name'].values)]
  print('files after:',all_data.shape[0])
  all_data = all_data.reset_index(drop=True)
  return all_data

# create API calls corpus
def create_API_corpus(all_data):
  API_corpus = []
  for file in all_data['file_name']:
      seq = all_data[all_data['file_name']==file].API_sequence.values[0]
      API_corpus.extend(set(seq))
  API_corpus = list(set(API_corpus))
  print('API_corpus:',len(API_corpus))
  return API_corpus

## Representation Learning Model

### Functions: one-hot-encoding


In [0]:
def convert_API_to_vector(Unique_API_calls,API):
    if API != 0:
        vec = np.zeros((len(Unique_API_calls),), dtype=np.float32)
        vec[Unique_API_calls.index(API)] = 1.0
    else:
        vec = np.zeros((len(Unique_API_calls),), dtype=np.float32)
    return list(vec)


def create_sequence_n_nextWord_one_hot(data):
    vectorized_data = []
    for idx in range(len(data)):
        seq = data.iloc[idx]
        vector = []
        for api in seq:
            vector.append(convert_API_to_vector(API_corpus,api))
        vectorized_data.append(vector)
    
    sequences = []
    next_words = []
    for seq in vectorized_data:
        sequences.append(seq[:-1])
        next_words.append(seq[len(seq)-1])
    
    return np.array(sequences),np.array(next_words)


def create_sequence_one_hot(data):
    vectorized_data = []
    for idx in range(len(data)):
        seq = data.iloc[idx]
        vector = []
        for api in seq:
            vector.append(convert_API_to_vector(API_corpus,api))
        vectorized_data.append(vector)
    
    return np.array(vectorized_data)


def pruning_n_padding(seq,sequence_MAX_length):
    final_seq = []
    if len(seq) < sequence_MAX_length+1:
        gap = (sequence_MAX_length) - len(seq)
        zeros = [0] * gap
        final_seq = list(zeros) + list(seq)
    
    elif len(seq) == sequence_MAX_length:
        final_seq = list(seq)
    
    else:
        final_seq = seq[:sequence_MAX_length]
    return final_seq



### Predict next API call 

#### Prepare the data

In [0]:
def prepare_data_nextWord(data,sequence_MAX_length):

  # pruning the sequences to match sequence_MAX_length + next API as a label
  LSTM_train_seq =  data.apply(lambda x: pruning_n_padding(x,sequence_MAX_length))
  # create sequences and nextWords
  train_vectorized_seq, train_vectorized_nextWord = create_sequence_n_nextWord_one_hot(LSTM_train_seq)

  # print('train_vectorized_seq:',train_vectorized_seq.shape, ' train_vectorized_nextWord:',train_vectorized_nextWord.shape)

  return train_vectorized_seq, train_vectorized_nextWord

#### Architecture 1:###
- Input: One-hot-encoding
- Train model with 2 LSTM layers + Dense using softmax
- Predict the next API call using the second layer last-cell (argmax)

**First stage - train LSTM**

In [0]:
def LSTM_nextWord_arch_1(output_dim,input_seq_size,input_time_dim,nb_units,dropout):
  seed_value= 10
  os.environ['PYTHONHASHSEED']=str(seed_value)
  random.seed(seed_value)
  np.random.seed(seed_value)
  # tf.set_random_seed(seed_value)
  tf.random.set_seed(seed_value)

  # build the model
  model_1 = Sequential()
  model_1.add(LSTM(units=nb_units, input_shape=(input_time_dim,input_seq_size),
                  activation='tanh',dropout=dropout ,return_sequences=True))
  model_1.add(LSTM(units=nb_units, input_shape=(input_time_dim,input_seq_size),
                  activation='tanh',dropout=dropout,return_sequences=False))
  model_1.add(Dense(output_dim,activation='softmax'))
  model_1.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  # print(model_1.summary())
  return model_1

**Evaluate the LSTM training**

In [0]:
def evaluate_LSTM_nextWord(model,x_test,y_test):
  predictions = model.predict(x_test)
  decoded_preds = []
  for idx in range(predictions.shape[0]):
      decoded = API_corpus[np.argmax(predictions[idx])]
      decoded_preds.append(decoded)
      
  decoded_true = []
  for idx in range(y_test.shape[0]):
      decoded = API_corpus[np.argmax(x_test[idx])]
      decoded_true.append(decoded)

  print('predicted corpus:', str(set(decoded_preds)))

  correct = 0
  for idx in range(len(decoded_preds)):
      if decoded_preds[idx] == decoded_true[idx]:
          correct += 1
          
  print('LSTM representation accuracy:', round(correct/len(decoded_preds),3))

#### Architecture 2: (Optional)###
- Input: integers
- Train model with 2 LSTM layers + Dense using softmax
- Predict the next API call using the second layer last-cell (argmax)

#### Architecture 3: (Optional)###
- Input: One-hot-encoding
- Train model with 1 LSTM layers (return_sequence=True)+ Dense using softmax
- Feed the model with the next API for each input cell (offset of the APIs)
- Next, remove the last Dense layer and perform GlobalMaxPooling for the representation

### Predict final label (0/1)

#### Prepare the data

In [0]:
def prepare_data_finalLabel(x_data, y_data,sequence_MAX_length):
  # print('Preparing data - final Label LSTM')

  # pruning the sequences to match sequence_MAX_length
  LSTM_train_seq =  x_data.apply(lambda x: pruning_n_padding(x,sequence_MAX_length))

  # convert API sequence to one-hot-encoding vectors
  train_vectorized_seq = create_sequence_one_hot(LSTM_train_seq)

  # convert class to 0 and 1
  rep_train_labels = np.array(y_data.apply(lambda x: 1 if x == 'malicious' else 0).values,dtype=np.float32)

  # print('x_data shape:',train_vectorized_seq.shape, ' y_data shape:',rep_train_labels.shape)

  return np.array(train_vectorized_seq), np.array(rep_train_labels)


#### Architecture 1:###
- Train model with 2 LSTM layers + Dense using sigmoid
- Predict the class using the second layer last-cell 
- Next, remove the second LSTM layer and perform GlobalMaxPooling for the representation

**First stage - train LSTM**

In [0]:
def LSTM_finalLabel_arch_1(input_seq_size,input_time_dim,nb_units,dropout):
  # Seed value
  seed_value= 10
  os.environ['PYTHONHASHSEED']=str(seed_value)
  random.seed(seed_value)
  np.random.seed(seed_value)
  # tf.set_random_seed(seed_value)
  tf.random.set_seed(seed_value)

  print('Bulding LSTM model - final Label (arch 1)')

  # build the model
  model_2 = Sequential()
  model_2.add(LSTM(units=nb_units, input_shape=(input_time_dim,input_seq_size),
                  activation='tanh',dropout=dropout ,return_sequences=True))
  model_2.add(LSTM(units=nb_units, input_shape=(input_time_dim,input_seq_size),
                  activation='tanh',dropout=dropout,return_sequences=False))
  model_2.add(Dense(1, activation='sigmoid'))
  model_2.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
  # print(model_2.summary())
  return model_2

**Evaluate the LSTM training**

In [0]:
def evaluate_LSTM_finalLabel(model,x_test,y_test):

  predictions = model.predict(x_test)
  final_preds = []
  for pred in predictions:
      if pred > 0.5:
          final_preds.append(1.0)
      else:
          final_preds.append(0.0)
          
  correct = 0
  for idx, pred in enumerate(final_preds):
      if final_preds[idx] == y_test[idx]:                            
          correct+=1
  print('LSTM representation accuracy:',round(correct/len(final_preds),4))

## Feature Extraction - from LSTM

### Extract features using LSTM

In [0]:
def feature_vector_maxPool_n_lastCell(trained_model, sequence_data):
  # Seed value
  seed_value= 10
  os.environ['PYTHONHASHSEED']=str(seed_value)
  random.seed(seed_value)
  np.random.seed(seed_value)
  # tf.set_random_seed(seed_value)
  tf.random.set_seed(seed_value)

  # remove the last two layers: Dense + last LSTM 
  new_model = Sequential()
  for layer in trained_model.layers[:-2]:
      new_model.add(layer)

  preds = new_model.predict(sequence_data)

  # save all the last lstm cell outputs
  last_cell_outputs = []
  for file in preds:
      last_cell_outputs.append(file[-1]) 

  del preds

  # remove the last two layers: Dense + last LSTM 
  new_model_max_pooling = Sequential()
  for layer in trained_model.layers[:-2]:
      new_model_max_pooling.add(layer)

  # add softmax + max pooling layer
  new_model_max_pooling.add(GlobalMaxPooling1D())
  max_pooling_vectors = new_model_max_pooling.predict(sequence_data)

  
  # Creating the representation vectors:
  # concatenating the max-pooling with the last output
  final_representation = []
  for idx,max_pool in enumerate(max_pooling_vectors):
      new_vector = list(max_pool) + list(last_cell_outputs[idx])
      final_representation.append(new_vector)
  final_representation = np.array(final_representation,dtype=np.float32)
  # print('final shape:', str(final_representation.shape) )
  del max_pooling_vectors
  
  return final_representation


In [0]:
def final_feature_vector(trained_model,x_data,y_data,one_hot_encoding,sequence_MAX_length):
  # pruning the sequences to match sequence_MAX_length 
  rep_train_seq =  x_data.apply(lambda x: pruning_n_padding(x,sequence_MAX_length))

  # convert class to 0 and 1
  labels = y_data.apply(lambda x: 1 if x == 'malicious' else 0).values

  if one_hot_encoding:
    # print('creating one-hot-encoding')
    # create sequences (one-hote encoding format) *we dont use the nextWord here
    train_vectorized_seq = create_sequence_one_hot(rep_train_seq)
    del rep_train_seq
    # print('Creating feature vectors')
    train_feature_vector = feature_vector_maxPool_n_lastCell(trained_model, train_vectorized_seq)
    del train_vectorized_seq

  return np.array(train_feature_vector), np.array(labels, dtype=np.float32)

## Classification

In [0]:
def calc_metrics(conf_matrix, Y_proba, Y_test):
    AUC = roc_auc_score(Y_test, Y_proba)

    TP = conf_matrix[1][1]
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]

    TPR = TP / (TP + FN)
    FPR = FP / (FP + TN)
    ACC = (TP + TN) / (TP + TN + FP + FN)
    Precision = TP / (TP + FP)
    F1_score = 2 * ((Precision * TPR) / (Precision + TPR))

    metrics_dict = {
        'TPR': round(TPR, 2),
        'FPR': round(FPR, 2),
        'ACC': round(ACC, 2),
        'Precision': round(Precision, 2),
        'F1_score': round(F1_score, 2),
        'AUC': round(AUC, 4),
    }

    return metrics_dict


def convert_class_to_binary(label):
    if label == 'malicious':
        return 1
    else:
        return 0



def get_predictions(Y_proba, threshold):
    Y_pred_final = []
    for p in Y_proba:
        if p >= threshold:
            Y_pred_final.append(1.0)
        else:
            Y_pred_final.append(0.0)

    return np.array(Y_pred_final)

def LR_fit(x_train, y_train, max_iter):

  # normalizing the data
  x_train_final = StandardScaler().fit_transform(x_train)
  # init model + fit
  LR = LogisticRegression(max_iter=max_iter)
  LR.fit(x_train_final, y_train)

  return LR


def LR_predict(model,x_test):
  # Get probability for each class
  x_test_final = StandardScaler().fit_transform(x_test)
  y_proba = model.predict_proba(x_test_final)[:, 1]
  # convert probabilities into predictions based on the threshold
  y_pred = get_predictions(y_proba, threshold=0.5)

  return y_pred,y_proba

def LSTM_predict(model, x_test):
  # Get probability for each class
  y_proba = model.predict(x_test)
  # convert probabilities into predictions based on the threshold
  y_pred = get_predictions(y_proba, threshold=0.5)

  return y_pred,y_proba


def print_final_results(y_test,folds_TPR,folds_FPR,folds_ACC,folds_Precision,folds_F1_score,folds_AUC):
  print('\nFinal Results: {} samples'.format(y_test.shape[0]))
  print('TPR:', round(np.mean(folds_TPR),4))
  print('FPR:', round(np.mean(folds_FPR),4))
  print('ACC:', round(np.mean(folds_ACC),4))
  print('Precision:', round(np.mean(folds_Precision),4))
  print('F1_score:', round(np.mean(folds_F1_score),4))
  print('AUC:', round(np.mean(folds_AUC),4))



# Main



## Load Train Data


In [18]:
all_data = pd.read_pickle('/content/drive/My Drive/Thesis/RNN_LSTM/data_for_LSTM.pkl')

# Clean data (remove short sequence & remove duplications)
sequence_MIN_length = 15 # (Pascano, 2015)
all_data = data_cleaning(sequence_MIN_length,all_data)

# use file from other experiments
dataset_name = '1_Balanced_Frequencies_first1000sec_minAPI_15_minRunTime_0_withYara_No.csv'
dataset_name = '0_Balanced_Frequencies_first1000sec_minAPI_0_minRunTime_0_withYara_No.csv'
folder_path = '/content/drive/My Drive/Thesis/RNN_LSTM/balaned_datasets'
full_path = '{}/{}'.format(folder_path,dataset_name)
all_data = use_spesific_files(full_path,all_data)

API_corpus = create_API_corpus(all_data)

all_data_distribuation = get_classes_distribution_df(all_data)
print(all_data_distribuation)


train_sequences = all_data['API_sequence']
train_labels = all_data['class']

Remove files with short sequence
files before: 7156
files after: 6166
files before: 6166
files after: 4596
API_corpus: 302
Benign:2277 (49.54%) | Malicious:2319 (50.46%)


## Set Params

In [0]:
# prepare data for representation model (Optional)
LSTM_train_seq_MAX_length = 50 # (Athiwaratkun, 2017)
LSTM_train_seq_MAX_length = 100 # (Pascanu, 2015)
LSTM_train_seq_MAX_length = 100 # (Pascanu, 2015)

# LSTM learning params
output_dim = len(API_corpus)
input_seq_size = len(API_corpus)
input_time_dim = None
nb_units = 1500 # (pascanu 2015, Agrawal 2018, Athiwaratkun 2017)
nb_units = 100 # (Zhang 2020)
dropout = 0.1
epochs = 5
# nb_folds = 10

representation_learning = False

# Classification params
classification_seq_MAX_length = 200 #(Agrawal 2018 , Athiwaratkun 2017)
one_hot_encoding = True



## Train model

In [0]:
# prepare data for LSTM training with the final label (0/1)
x_train,y_train = prepare_data_finalLabel(train_sequences,train_labels,LSTM_train_seq_MAX_length)


# create the LSTM model
LSTM_model = LSTM_finalLabel_arch_1(input_seq_size,
                                    input_time_dim,
                                    nb_units,
                                    dropout)

# train model on training data
LSTM_model.fit(x_train,y_train, epochs=epochs)

LSTM_model.save('/content/drive/My Drive/Thesis/RNN_LSTM/Trained_LSTM_finalLabel_arch_1.h5')

Bulding LSTM model - final Label (arch 1)
Epoch 1/5
4596/4596 [==============================] - 50s 11ms/step - loss: 0.4852 - accuracy: 0.7846
Epoch 2/5
4596/4596 [==============================] - 48s 10ms/step - loss: 0.3719 - accuracy: 0.8527
Epoch 3/5
4596/4596 [==============================] - 48s 10ms/step - loss: 0.3291 - accuracy: 0.8747
Epoch 4/5
4596/4596 [==============================] - 48s 10ms/step - loss: 0.3021 - accuracy: 0.8864
Epoch 5/5
4596/4596 [==============================] - 49s 11ms/step - loss: 0.3001 - accuracy: 0.8893


## Load Test Data

In [31]:
test_data = load_raw_parquets('/content/drive/My Drive/Thesis/RNN_LSTM/Detection_data/APT_raw_parquets')
test_data = data_cleaning(1,test_data)

test_sequences = test_data['API_sequence']
test_labels = test_data['class']


# prepare data for LSTM training with the final label (0/1)
x_test, y_test = prepare_data_finalLabel(test_sequences,test_labels,LSTM_train_seq_MAX_length)

batch 1  loaded
Remove files with short sequence
files before: 6
files after: 6


## Detection - Only LSTM

In [27]:
LSTM_model_trained = load_model('/content/drive/My Drive/Thesis/RNN_LSTM/Trained_LSTM_finalLabel_arch_1.h5')


y_preds,y_proba = LSTM_predict(LSTM_model_trained,x_test)
y_proba

array([[0.99752945],
       [0.75231177],
       [0.99350834],
       [0.7061012 ],
       [0.22993211],
       [0.75231177]], dtype=float32)

## Detection - LSTM + LR

In [0]:
LSTM_model_trained = load_model('/content/drive/My Drive/Thesis/RNN_LSTM/Trained_LSTM_finalLabel_arch_1.h5')

# create train feature vector - feature extraction
x_train_rep, y_train_rep = final_feature_vector(LSTM_model_trained,
                                    train_sequences,
                                    train_labels,
                                    one_hot_encoding,
                                    classification_seq_MAX_length)


In [0]:


# create train feature vector - feature extraction
x_train_rep, y_train_rep = final_feature_vector(LSTM_model_trained,
                                    train_sequences,
                                    train_labels,
                                    one_hot_encoding,
                                    classification_seq_MAX_length)

# create test feature vector - feature extraction
x_test_rep, y_test_rep = final_feature_vector(LSTM_model_trained,
                                    test_sequences,
                                    test_labels,
                                    one_hot_encoding,
                                    classification_seq_MAX_length)

# train classifier
LR_classifier = LR_fit(x_train_rep, y_train_rep, max_iter=1000)

# predict 
y_preds,y_proba = LR_predict(LR_classifier,x_test_rep)
y_proba

## Save Results

In [0]:
results = pd.DataFrame(columns=['file_name','true_class','LSTM'])

for idx, f in enumerate(test_data['file_name'].unique()):
  f_df = test_data[test_data['file_name'] == f]
  file_name = f_df['file_name'].values[0]
  file_class = int(y_test[idx])
  prediction = y_proba[idx][0]

  new_record = {
      'file_name':file_name,'true_class':file_class,'LSTM':prediction
  }

  results = results.append(new_record, ignore_index=True)

results.to_csv('/content/drive/My Drive/Thesis/RNN_LSTM/LSTM_results.csv', index=False)